In [1]:
from __future__ import print_function
from __future__ import division

import os
import time
import curses
import numpy as np
import cv2
import src.viztools as vv

 ### TODO
 * rgbburst style 2
 * refactor key input handling across classes (change phase keys in Alien)
 * border zoom out black masking

In [46]:
"""vid-viz main loop

KEYBOARD INPUTS:
    number keys - choose effect (see viztools.py for detailed keyboard input for each effect)
        0 - thresholding
        1 - alien
        3 - rgbwalk
        4 - rgbburst
    q - quit effect (then number keys are once again available for choosing a new effect)
    ` - enable toggling of border effects ('t' to toggle, tab to switch processing/border order)
    m - enable toggling of masking effects (tab to switch processing/masking order)
    spacebar - toggle between mask and no mask; if no mask has been specified, current frame is used
    backspace - quit border and masking effect editing
    esc - exit loop
"""

# general user parameters
SOURCE = 'image'      # 'cam' | 'video' | 'image'
DISP_FULL_SCREEN = 0
ANIM_STYLE = 'loop'   # 'loop' | 'pcycle' | 'rand' - reverse dir eats up lots of CPU in pcycle and rand
USE_MASK = False      # spacebar to toggle

# select video source
if SOURCE is 'cam':
    cap = cv2.VideoCapture(0)
    frame_mask = None
elif SOURCE is 'video':
#     cap = cv2.VideoCapture(0)
#     cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/tunnel_01.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-12-18 23-15-59.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-08-26 02-48-16.mp4')
    cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/snowflake_02.mp4')
    frame_mask = None
elif SOURCE is 'image':
    frame_orig = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/honeycomb_01.jpg')
    frame_orig = cv2.resize(
        frame_orig,
        None,
        fx=0.5,
        fy=0.5,
        interpolation=cv2.INTER_LINEAR)
    frame_mask = np.copy(frame_orig)    

# get video properties
if SOURCE is 'video':
    # target_fps = cap.get(cv2.CAP_PROP_FPS)
    tot_fr_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
else:
    tot_fr_count = float('inf')
target_fps = 20.0

# set standard options
ANIM_DIR = 'forward'
post_process = False # whether or not post-processing is active effect
post_process_pre = False # whether post-processing pre/proceeds other effects
mask_process = False
mask_process_pre = True

# initialize key handler
effect_index = None
key_list = [False for i in range(256)]

# initialize processing objects
effects = [
    vv.Threshold(),
    vv.Alien(),
    vv.RGBWalk(),
    vv.RGBBurst()]
num_effects = len(effects)
border = vv.Border()
masker = vv.Mask()

# for displaying program info
# stdscr = curses.initscr()
# curses.noecho()
# curses.cbreak()
    
fr_count = 0
while(True):

    time_pre = time.time()
    
    # get frame and relevant info
    if SOURCE is 'cam' or SOURCE is 'video':
        ret, frame = cap.read()
        # crop image to have same aspect ratio as monitor
        if SOURCE is 'cam':
            frame = frame[60:-60,:,:]
    else:
        # only update frame if not updating mask
        frame = np.copy(frame_orig)

    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    key_list[key] = True
    if key == 27:
        # escape key; exit program
        break
    elif key == ord('q'):
        # quit current effect
        effect_index = None
    elif key == ord('\b'):
        post_process = False
        mask_process = False
    elif key == ord('`'):
        post_process = True
    elif key == ord('m'):
        mask_process = True
    elif key == ord(' '):
        USE_MASK = not USE_MASK
    elif key == ord('\t'):
        # only change post-processing order if in post-process mode
        if post_process:
            post_process_pre = not post_process_pre
        elif mask_process:
            mask_process_pre = not mask_process_pre
#     print('\r%g' % key, end='')
    
    # update current effect
    if effect_index is None:
        for num in range(10):
            if key == ord(str(num)):
                effect_index = num
                key_list[key] = False

    # update mask
    if USE_MASK:
        if mask_process:
            mask = masker.process(frame_mask, key_list)
        else:
            mask = masker.process(frame_mask, key_list, key_lock=True)
            
    # apply mask before borders and effect
    if mask_process_pre and USE_MASK:
        frame = cv2.bitwise_and(frame, frame, mask=mask)
    
    # apply borders before effect
    if post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)
    
    # process frame
    if effect_index is not None and effect_index < num_effects:
        if post_process:
            frame = effects[effect_index].process(frame, key_list, key_lock=True)            
        else:
            frame = effects[effect_index].process(frame, key_list)

    # apply mask before borders and after effect
    if not mask_process_pre and USE_MASK:
        frame = cv2.bitwise_and(frame, frame, mask=mask)
            
    # apply borders after effect
    if not post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)

    # display frame
    if DISP_FULL_SCREEN:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        if SOURCE is 'cam':
            frame = cv2.resize(frame, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    # control animation
    if ANIM_STYLE is 'loop':
        fr_count += 1
        if fr_count == tot_fr_count:
            # reset frame postion to 1 (not zero so window isn't moved)
            cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
            fr_count = 1
    elif ANIM_STYLE is 'pcycle':
        # update frame index
        if ANIM_DIR is 'forward':
            fr_count += 1
        elif ANIM_DIR is 'backward':
            fr_count -= 1
        # reverse direction if necessary
        if fr_count == tot_fr_count:
            fr_count -= 2
            ANIM_DIR = 'backward'
        elif fr_count == 0:
            fr_count = 1
            ANIM_DIR = 'forward'
            cap.set(cv2.CAP_PROP_POS_FRAMES, fr_count)
        # update
        if ANIM_DIR is 'backward':
            cap.set(cv2.CAP_PROP_POS_FRAMES, fr_count)

    # calculate, limit and output fps
    time_tot = time.time() - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_tot = time.time() - time_pre
    print('\r%03i fps' % (1.0 / time_tot), end='')
#     stdscr.addstr(0, 0, '%03i fps' % (1.0 / time_tot))
#     stdscr.addstr(1, 0, '%05i frames' % fr_count)
#     stdscr.refresh()
    

    # reset key list (for pressed keys that are not reset by effect object(s))
    key_list[key] = False
    
if SOURCE is not 'image':
    cap.release()
cv2.destroyWindow('frame')

019 fps

In [38]:
reload(vv)

<module 'src.viztools' from 'src/viztools.py'>

In [ ]:
%timeit cap.set(cv2.CAP_PROP_POS_FRAMES,1)

In [ ]:
img1 = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')

# img0 = 0
# img2 = cv2.addWeighted(img0, 0.1, img1, 0.9, 0)
# %timeit cv2.addWeighted(img0, 0.1, img1, 0.9, 0)

# im_width, im_height, _ = img1.shape
# %timeit cv2.warpAffine(img1, \
#             np.float32([[1, 0, 100], \
#                         [0, 1, 200]]), \
#             (im_width, im_height))

%timeit cv2.resize(img1, None, fx=2, fy=2, interpolation = cv2.INTER_LINEAR)

In [ ]:
img1 = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')

while(True):
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    print('\r%g' % key, end='')
    
    if key == 27:
        # escape key
        break
        
        cv2.imshow('frame', img1)
        
cap.release()
cv2.destroyWindow('frame')

### To patrol-cycle an mp4 with ffmpeg
* ffmpeg -i source.mp4 -vf reverse source_rev.mp4
* ffmpeg -f concat -i concat_list.txt -c copy concated.mp4

##### concat_list.txt
file '/path/to/vid0.mp4'

file '/path/to/vid1.mp4'